# Datatypes

The following table lists all synthesizable data types and their VHDL equivalent. Synthesizable types can be wrapped in type qualifies to produce runtime variable objects.

| CoHDL       | VHDL               |
|-------------|--------------------|
| `Bit`       | `std_logic`        |
| `BitVector` | `std_logic_vector` |
| `Unsigned`  | `unsigned`         |
| `Signed`    | `signed`           |
| `int`       | `integer`          |
| `bool`      | `boolean`          |
| `Enum`      | `enumeration`      |

---
## Bit

The `Bit` type is equivalent to VHDLs `std_logic`.

In [ ]:
from cohdl import Entity, Port, Bit, BitState, Null, Full
from cohdl import std

class BitExample(Entity):

    a = Port.output(Bit)
    b = Port.output(Bit)
    c = Port.output(Bit)
    d = Port.output(Bit)
    e = Port.output(Bit)
    f = Port.output(Bit)
    g = Port.output(Bit)
    h = Port.output(Bit)

    x = Port.output(Bit)
    y = Port.output(Bit)
    z = Port.output(Bit)

    def architecture(self):

        @std.concurrent
        def logic():

            # different ways to assign to a Bit signal

            self.a <<= "0"
            self.b <<= "1"
            self.c <<= False
            self.d <<= True
            self.e <<= Null
            self.f <<= Full
            self.g <<= BitState.LOW
            self.h <<= BitState.HIGH
        
        @std.concurrent
        def logic_operators():
            # Bit implements the basic binary operations
            # and, or and xor
            self.x <<= self.a & self.b
            self.y <<= self.a | self.b
            self.z <<= self.a ^ self.b
        

print(std.VhdlCompiler.to_string(BitExample))

---
## vector types

CoHDL defines three vector types (`BitVector`, `Unsigned` and `Signed`). Each consists of a fixed number of Bits, they differ only in the way arithmetic operators are implemented for them.

The following example shows, how the width of the vector types `BitVector`, `Signed` and `Unsigned` is defined. Currently only bitorder 'downto' is supported.

In [ ]:
from cohdl import BitVector, Signed, Unsigned

bv_a = BitVector[7:0]
bv_b = BitVector[8]   # equivalent to [7:0]

s_a = Signed[4:0]
s_b = Signed[5]       # equivalent to [4:0]

u_a = Unsigned[15:0]
u_b = Unsigned[16]    # equivalent to [15:0]

### assigning and casting vector types

The different vector types can be assigned to each other, when the assignment is unambiguous and no bits are lost. The following table lists under which conditions a source value of width `S` can be assigned to a target of width `T`.

|source\target|BitVector|Unsigned|Signed|
|-------------|---------|--------|------|
|BitVector    | S == T  | S == T |S == T|
|Unsigned     | S == T  | S <= T |S <  T|
|Signed       | S == T  |   -    |S <= T|

The three vector types can be converted into each other using the properties `bitvector`, `signed` and `unsigned`.


In [ ]:
from cohdl import Entity, Port, Signal, Unsigned, Signed, BitVector
from cohdl import std

class TypeCasts(Entity):
    output_v = Port.output(BitVector[16])
    output_u = Port.output(Unsigned[16])
    output_s = Port.output(Signed[16])

    def architecture(self):

        # local signals, only 8 bit wide to
        # demonstrate automatic resize when assigned
        # to wider numeric types
        local_v = Signal[BitVector[8]]()
        local_u = Signal[Unsigned[8]]()
        local_s = Signal[Signed[8]]()
        
        @std.concurrent
        def logic():
            self.output_u <<= local_u

            # The BitVector local_v is cast to Unsigned
            # before the assignment to output_u.
            self.output_u <<= local_v.unsigned

            # Both signed and unsigned values are assignable
            # to wider signed objects. Plain BitVectors
            # must be cast before assignment.
            self.output_s <<= local_s
            self.output_s <<= local_u
            self.output_s <<= local_v.signed

            # Type casts can also appear on the left hand side of assignments
            self.output_v.unsigned <<= local_u
            self.output_v.signed <<= local_s

print(std.VhdlCompiler.to_string(TypeCasts))

### slices

CoHDL uses the `[]`-Operator to extract bits and slices from vectors. The return value of these expressions is a reference to the specified bits and can be used as an alias for parts of vectors. This works both in synthesizable and non-synthesizable contexts.

In [ ]:
from cohdl import Entity, Port, BitVector
from cohdl import std

class ExampleSlices(Entity):
    input_v = Port.output(BitVector[16])
    output_v = Port.output(BitVector[16])

    def architecture(self):
        @std.concurrent
        def logic_1():
            self.output_v[15] <<= self.input_v[0]

            self.output_v[10:5].unsigned <<= self.input_v[5:0].unsigned + 1

        # CoHDL tracks vector slices even when they are defined outside
        # synthesizable contexts. Here input_bit and output_bit are defined
        # as aliases for bit 0 of input_v and bit 15 of output_v.
        input_bit = self.input_v[0]
        output_bit = self.output_v[15]

        # slice aliases can also be type cast
        input_slice = self.input_v[5:0].unsigned
        output_slice = self.output_v[10:5].unsigned

        # equivalent to logic_1 but using aliases
        # instead of explicit slices
        @std.concurrent
        def logic_2():
            output_bit.next = input_bit
            output_slice.next = input_slice + 1
        

print(std.VhdlCompiler.to_string(ExampleSlices))

The following example shows, how vector slices can be used in combination with Python classes to provide aliases for fields in a configuration register.

In [ ]:
from cohdl import Entity, Port, Signal, Bit, Unsigned, BitVector
from cohdl import std

class Config:
    def __init__(self, raw: Signal[BitVector]):
        self.enable = raw[0]
        self.cnt_down = raw[1]
        self.prescaler = raw[31:16].unsigned


class Counter(Entity):
    clk = Port.input(Bit)

    config = Port.input(BitVector[32])
    output = Port.output(Unsigned[32])

    def architecture(self):
        clk = std.Clock(self.clk)

        # wrap the config bitvector in a Python class that provides
        # aliases to the contained register fields
        cfg = Config(self.config)

        # tick is high for one clock period when the prescaler runs out
        tick = Signal[Bit](False)
        cnt = Signal[Unsigned[16]](0)

        @std.sequential(clk)
        def process_prescaler():
            if cnt == 0:
                cnt.next = cfg.prescaler
                tick.push = True
            else:
                cnt.next = cnt - 1
        
        # increment or decrement the output counter
        # when the prescaler runs out
        @std.sequential(clk)
        def process_output():
            if tick:
                if cfg.cnt_down:
                    self.output <<= self.output - 1
                else:
                    self.output <<= self.output + 1

print(std.VhdlCompiler.to_string(Counter))

## Null and Full

CoHDL defines two singleton objects `cohdl.Null` and `cohdl.Full` as a replacement for VHDLs `(others => 'x')` construct. When assigned to vectors they set all bits to `'0'` or `'1'`.

In [ ]:
from cohdl import Entity, Port, BitVector, Bit, Null, Full
from cohdl import std

class ExampleNullFull(Entity):
    a = Port.output(BitVector[1])
    b = Port.output(BitVector[2])
    c = Port.output(BitVector[3])
    d = Port.output(BitVector[4])

    x = Port.output(Bit)

    def architecture(self):
        @std.concurrent
        def logic():
            self.a <<= Null
            self.b <<= Full
            self.c <<= Null
            self.d <<= Full

            # assigning Null and Full to bits works too
            self.x <<= Null
        
print(std.VhdlCompiler.to_string(ExampleNullFull))

---
## Enums

CoHDL provides the submodule `cohdl.enum` that implements part of Pythons standard `enum` module. Types derived from `cohdl.enum.Enum` are synthesizable (can be wrapped in Signals, Variables and Temporaries). As the following example shows CoHDL turns them into VHDL enumeration types.

In [ ]:
from cohdl import Entity, Port, Signal, Bit, Unsigned, enum
from cohdl import std

class MyEnum(enum.Enum):
    first = enum.auto()
    second = enum.auto()
    third = enum.auto()

class EnumExample(Entity):
    clk = Port.input(Bit)

    output = Port.output(Unsigned[8])

    def architecture(self):
        state = Signal[MyEnum](MyEnum.first)

        @std.sequential(std.Clock(self.clk))
        def logic_a():
            nonlocal state

            # cohdl supports match statements
            # (but no pattern matching)
            match state:
                case MyEnum.first:
                    state <<= MyEnum.second
                    self.output <<= 1
                case MyEnum.second:
                    state <<= MyEnum.third
                    self.output <<= 2
                case MyEnum.third:
                    state <<= MyEnum.first
                    self.output <<= 3
        

print(std.VhdlCompiler.to_string(EnumExample))

---
## Arrays

Array types are defined using the expression `cohdl.Array[DATA_TYPE, CNT]` where `DATA_TYPE` is a synthesizable datatype and `CNT` is a positive integer.

The builtin array type `cohdl.Array` only supports primitive element types (Bit, BitVectors...). The standard library provides `std.Array` as an abstraction that can hold any serializable datatype (see the section on user defined data types).

In [1]:
from cohdl import Array, Bit

# define a new array type 'Array[Bit, 16]'
my_array_type = Array[Bit, 16]

# define a new object 'my_array' of type 'Array[Bit, 16]'
my_array = Array[Bit, 16]()
my_array = my_array_type()  # equivalent to previous line

In [ ]:
from cohdl import Entity, Port, Signal, Bit, Unsigned, Array, BitVector
from cohdl import std


class ArrayExample(Entity):
    clk = Port.input(Bit)

    addr_in = Port.input(Unsigned[4])
    data_in = Port.input(BitVector[8])

    addr_out = Port.input(Unsigned[4])
    data_out = Port.output(BitVector[8])

    def architecture(self):
        clk = std.Clock(self.clk)

        # like all other synthesizable types array types
        # must be type qualified to create runtime variable objects
        memory = Signal[Array[Unsigned[8], 16]]()

        @std.sequential(clk)
        def process_write():
            # array elements are accessed using the []-operator
            # the arguments type must be one of int, Signed or Unsigned
            memory[self.addr_in] <<= self.data_in
        
        @std.sequential(clk)
        def process_read():
            self.data_out <<= memory[self.addr_out]
        
print(std.VhdlCompiler.to_string(ArrayExample))